In [2]:
import cv2
import os
import numpy as np
import pandas as pd
import tensorflow as tf
import matplotlib.pyplot as plt
from PIL import Image
from sklearn.model_selection import train_test_split
from keras import layers, Input, models, optimizers
from keras.models import Model, Sequential
from keras.layers import MaxPooling2D, Conv2D
from keras.layers import Activation, Dropout, Flatten, Dense
from keras.callbacks import ModelCheckpoint
from keras.layers.normalization import BatchNormalization
from keras.preprocessing.image import ImageDataGenerator

In [3]:
model = Sequential()
model.add(Conv2D(16, (3, 3), padding='same', use_bias=False, input_shape=(150,150, 3))) # RGB:3, gray:1
model.add(BatchNormalization(axis=3, scale=False))
model.add(Activation("relu"))
model.add(MaxPooling2D(pool_size=(4, 4), strides=(4, 4), padding='same'))
#model.add(Dropout(0.2))
 
model.add(Conv2D(32, (3, 3), padding='same', use_bias=False))
model.add(BatchNormalization(axis=3, scale=False))
model.add(Activation("relu"))
model.add(MaxPooling2D(pool_size=(4, 4), strides=(4, 4), padding='same'))
#model.add(Dropout(0.2))
 
model.add(Conv2D(64, (3, 3), padding='same', use_bias=False))
model.add(BatchNormalization(axis=3, scale=False))
model.add(Activation("relu"))
model.add(MaxPooling2D(pool_size=(4, 4), strides=(4, 4), padding='same'))
#model.add(Dropout(0.2))
 
model.add(Conv2D(128, (3, 3), padding='same', use_bias=False))
model.add(BatchNormalization(axis=3, scale=False))
model.add(Activation("relu"))
model.add(Flatten())
#model.add(Dropout(0.2))
 
model.add(Dense(512, activation='relu'))
model.add(Dense(7, activation='softmax'))                                   # label, calss 갯수
model.summary()
 

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 150, 150, 16)      432       
_________________________________________________________________
batch_normalization_1 (Batch (None, 150, 150, 16)      48        
_________________________________________________________________
activation_1 (Activation)    (None, 150, 150, 16)      0         
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 38, 38, 16)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 38, 38, 32)        4608      
_________________________________________________________________
batch_normalization_2 (Batch (None, 38, 38, 32)        96        
_________________________________________________________________
activation_2 (Activation)    (None, 38, 38, 32)        0         
__________

## compile 및 데이터 불러오기

In [4]:
# Compiling the CNN

from keras.optimizers import Adam
optimizer = Adam(lr=0.001)

model.compile(optimizer = optimizer, 
              loss = 'categorical_crossentropy', 
              metrics = ['accuracy'])


# 데이터 셋
train_datagen = ImageDataGenerator(rescale=1./255,
#                                    rotation_range=10,
#                                    width_shift_range=0.2,
#                                    height_shift_range=0.2,
#                                    shear_range=0.7,
#                                    zoom_range=[0.9, 2.2],
#                                    horizontal_flip=True,
#                                    vertical_flip=True,
#                                    fill_mode='nearest',
                                   validation_split=0.33)
 
training_set = train_datagen.flow_from_directory('C:/python_DA/final_project/img/pattern_original',
                                                 target_size = (150,150),
                                                 batch_size = 10,
                                                 class_mode = 'categorical',
                                                 subset="training"
#                                                 ,color_mode='grayscale'            # gray_scale로 변경
                                                )
validation_set = train_datagen.flow_from_directory('C:/python_DA/final_project/img/pattern_original',
                                                 target_size = (150,150),
                                                 batch_size = 10,
                                                 class_mode = 'categorical',
                                                 subset="validation"
#                                                 ,color_mode='grayscale'           # gray_scale로 변경
                                                  )



Found 4177 images belonging to 7 classes.
Found 2055 images belonging to 7 classes.


## 모델 학습

In [5]:
from keras.callbacks import EarlyStopping
from keras.callbacks import ModelCheckpoint


# moniter: 대상 값, mode: min/max, verbose=1 설정시 언제 멈췄는지 출력
# patience: 성능없는 에폭 허용 범위, baseline: 값 설정

#es = EarlyStopping(monitor='val_loss', mode='min')
#es = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=50)

In [6]:
es = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=30)
mc = ModelCheckpoint('best0005.h5', monitor='val_loss', mode='min', save_best_only=True)

In [ ]:
hist = model.fit_generator(training_set,
                         steps_per_epoch = 40, #20
                         epochs = 100,  #1000
                         validation_data = validation_set,
                         validation_steps = 20,
                         callbacks=[es,mc])
 

Epoch 1/100
40/40 [==============================] - 33s 818ms/step - loss: 2.2023 - acc: 0.2400 - val_loss: 1.9662 - val_acc: 0.2600
Epoch 2/100
40/40 [==============================] - 33s 821ms/step - loss: 1.8657 - acc: 0.3375 - val_loss: 1.6543 - val_acc: 0.4150
Epoch 3/100
40/40 [==============================] - 32s 789ms/step - loss: 1.5112 - acc: 0.4400 - val_loss: 1.9117 - val_acc: 0.3300
Epoch 4/100
40/40 [==============================] - 32s 796ms/step - loss: 1.3952 - acc: 0.4825 - val_loss: 1.4684 - val_acc: 0.4200
Epoch 5/100
40/40 [==============================] - 32s 800ms/step - loss: 1.4178 - acc: 0.4900 - val_loss: 1.5907 - val_acc: 0.3650
Epoch 6/100
40/40 [==============================] - 31s 779ms/step - loss: 1.2255 - acc: 0.5475 - val_loss: 1.7086 - val_acc: 0.4350
Epoch 7/100
40/40 [==============================] - 31s 778ms/step - loss: 1.3082 - acc: 0.5203 - val_loss: 2.6795 - val_acc: 0.3600
Epoch 8/100
40/40 [==============================] - 32s 788ms

## 학습 결과 확인: 그래프

In [ ]:
# 학습 과정 정확도, 손실함수 변화 그래프로 보기

import matplotlib.pyplot as plt
 
fig, loss_ax = plt.subplots()
acc_ax = loss_ax.twinx()
 
#loss_ax.plot(hist.history['loss'], 'y', label='train loss')
loss_ax.plot(hist.history['val_loss'], 'r', label='val loss')
##loss_ax.set_ylim([0.0, 0.5])
 
#acc_ax.plot(hist.history['acc'], 'b', label='train acc')
acc_ax.plot(hist.history['val_acc'], 'g', label='val acc')
##acc_ax.set_ylim([0.8, 1.0])
 
loss_ax.set_xlabel('epoch')
loss_ax.set_ylabel('loss')
acc_ax.set_ylabel('accuray')
 
loss_ax.legend(loc='upper left')
acc_ax.legend(loc='lower left')
 
plt.show()

In [ ]:
# 학습 과정 정확도, 손실함수 변화 그래프로 보기

import matplotlib.pyplot as plt
 
fig, loss_ax = plt.subplots()
acc_ax = loss_ax.twinx()
 
loss_ax.plot(hist.history['loss'], 'y', label='train loss')
#loss_ax.plot(hist.history['val_loss'], 'r', label='val loss')
##loss_ax.set_ylim([0.0, 0.5])
 
acc_ax.plot(hist.history['acc'], 'b', label='train acc')
#acc_ax.plot(hist.history['val_acc'], 'g', label='val acc')
##acc_ax.set_ylim([0.8, 1.0])
 
loss_ax.set_xlabel('epoch')
loss_ax.set_ylabel('loss')
acc_ax.set_ylabel('accuray')
 
loss_ax.legend(loc='upper left')
acc_ax.legend(loc='lower left')
 
plt.show()

In [ ]:
# 다른 방식으로 나타낸 그래프

acc = hist.history['acc']
val_acc = hist.history['val_acc']
loss = hist.history['loss']
val_loss = hist.history['val_loss']

epochs = range(len(acc))

plt.plot(epochs, acc, 'bo', label='Training acc')
plt.plot(epochs, val_acc, 'b', label='Validation acc')
plt.title('Training and validation accuracy')
plt.legend()

plt.figure()

plt.plot(epochs, loss, 'bo', label='Training loss')
plt.plot(epochs, val_loss, 'b', label='Validation loss')
plt.title('Training and validation loss')
plt.legend()

plt.show()

## 학습 결과 확인: 정확도 및 예측

In [ ]:
# output = classifier.predict_generator(test_set, steps=5)
# print(test_set.class_indices)
# print(output)
 
# 모델 평가하기
print("-- Evaluate : training acc --") 
train_scores = model.evaluate_generator(
               training_set,
               steps = 10) 
print("%s: %.2f%%" %(model.metrics_names[1], train_scores[1]*100))
print('\n')


print("-- Evaluate : validation acc --") 
vali_scores = model.evaluate_generator(
            validation_set,
            steps = 10) 
print("%s: %.2f%%" %(model.metrics_names[1], vali_scores[1]*100))
 
    
    
# 모델 예측하기
# print("-- Predict --")
 
# output = model.predict_generator(
#             validation_set,
#             steps = 100)
# print(validation_set.class_indices)
# np.set_printoptions(formatter={'float': lambda x: "{0:0.3f}".format(x)})
# print(output) 

## 모델 저장

In [ ]:
# 모델 저장


from keras.models import load_model, model_from_json
model.save('pattern_7class_정확도77.h5')                      # 모델(architecture, weight 동시 저장) 저장 이름


# 모델 아키텍처를 json 형, 식으로 저장                 # 모델 architecture 만 저장 
#json_string = model.to_json()                       # json 파일에서 모델 아키텍처 재구성
#model = model_from_json(json_string) 


# 모델 Weights 저장
# model.save_weights('파일이름')


## 저장 모델 load 및 예측

In [ ]:
from keras.models import load_model

model = load_model('pattern_7class_정확도77.h5') # 불러올 모델 이름

# print("-- Evaluate : validation acc --") 
# vali_scores = model.evaluate_generator(
#             validation_set) 
# print("%s: %.2f%%" %(model.metrics_names[1], vali_scores[1]*100))
 